# Basic setup

Here we will import the `pyspark` module and set up a `SparkSession`.  We'll use a `SparkSession` running locally, with one Spark executor for every core on the local machine.  Since we'll be accessing some data files that are only accessible from within the notebook container, we won't run against a cluster.


In [ ]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# Creating a resilient distributed dataset (RDD)

One of the easiest ways to create a resilient distributed dataset is from a local collection, with the `parallelize` method on the `SparkContext` object.

In [ ]:
numberRDD = sc.parallelize(range(1, 10000))

You can also create RDDs from files, S3 objects, and other external data sources.  See [the documentation](https://spark.apache.org/docs/latest/programming-guide.html#external-datasets) for more information. 

# Basic RDD transformations 

RDDs are _immutable_, so to transform a RDD, we create a new one.  RDDs are also _lazy_, so instead of transforming the elements when we create the new RDD, we store a reference to the original and the operation we'd need to apply to it to construct the transformed RDD.

In [ ]:
# filter numberRDD, keeping only even numbers
evens = numberRDD.filter(lambda x: x % 2 == 0)

# produce an RDD by doubling every element in numberRDD
doubled = numberRDD.map(lambda x: x * 2)

# filter numberRDD, keeping only multiples of five
fives = numberRDD.filter(lambda x: x % 5 == 0)

# return an RDD of the elements in both evens and fives
tens = evens.intersection(fives)
sortedTens = tens.sortBy(lambda x: x)

You can see other RDD transformations in the [Spark documentation](https://spark.apache.org/docs/latest/programming-guide.html#transformations).

# RDD actions

Since RDDs are lazy and RDD transformations don't actually compute anything, we need some way to force Spark to actually schedule a computation.  RDD _actions_ are operations that schedule the graph of computations implied by an RDD and return a result to the main program.  Here are a few examples:

In [ ]:
(evens.count(), doubled.count())

In [ ]:
# note that we may not get results in order!
tens.take(5)

In [ ]:
# ...unless we sort
sortedTens.take(5)

In [ ]:
# we can take a sample from an RDD (with or without replacement)
sortedTens.takeSample(False, 10)

In [ ]:
sortedTens.reduce(lambda x, y: max(x, y))

You can see some other RDD actions in the [Spark documentation](https://spark.apache.org/docs/latest/programming-guide.html#actions).

# Structured query and data frames

Spark also includes support for structured queries, including SQL and pandas- or R-like "data frame" operations through a query DSL.

Let's see structured query in action by loading a [Parquet](http://parquet.apache.org/) file with some simplified [fedmsg](https://fedora-fedmsg.readthedocs.io/en/latest/) log messages.

In [ ]:
df = spark.read.load("/data/msgs.parquet")
df.printSchema()

We can use the data frame DSL to do RDBMS-style queries on these data, which is great for characterizing or exploring it.  Because these queries can execute in parallel or across multiple machines, aggregations can be _much_ faster in Spark than they are on a traditional RDBMS.  

It is simple to do basic aggregations, like this:

In [ ]:
df.groupBy('category').count().orderBy('count', ascending=False).show()

(This file isn't huge, but it's in the Docker image.)

In [ ]:
df.count()

We can use the `show` method to quickly inspect a few rows of a data frame (not just the results of a query).  This is often helpful to sanity-check a new data source.

In [ ]:
df.show(10)

# Data cleaning

Uh oh!  It looks like the `msg` field of this data frame is JSON-encoded message structures instead of actual message structures.  While we'd _never_ see messy data in the real world, this really throws a wrench into our tutorial.  Let's fix that by asking Spark to infer a schema for the JSON fields.

In [ ]:
msgRDD = df.select("msg").rdd.map(lambda x: x[0])
# structs = sqlc.jsonRDD(msgRDD)
# structs.printSchema()

You'll notice that the last two lines are commented out there, and with good reason.  You can uncomment them and run them, but only if you're patient and willing to scroll. You'll get a huge schema with objects that have one field for (as one example) every Fedora user who has ever participated in an IRC meeting! (Alternatively, [click here](https://gist.github.com/willb/ede22cdcd25b64e8cda952f927701d96) to see a rendered version of the inferred schema.) 

Spark can't infer a useful schema for these JSON records, because their schemas diverge and because of some of the unusual ways that `fedmsg` data uses JSON to encode maps.  While there are a few reasons for the schema divergence (see [a practical treatment](http://chapeau.freevariable.com/2014/10/fedmsg-and-spark.html) or a more [type-theoretic one](http://chapeau.freevariable.com/2014/11/algebraic-types.html)), in this case one problem is that different `fedmsg` messages use the `branches` field to refer to values with incompatible types.

Fortunately, we can fix that with a pretty quick hack.  We'll just go through every record and retain only a few fields that we know are not going to give us grief.  (You'd probably want to do something more sophisticated in a real application.)  We'll use Spark's _user-defined function_ mechanism to achieve this.

In [ ]:
import json

# define the fields we want to keep
interesting_fields = ['agent', 'author', 'copr', 'user', 'msg', 'meeting_topic', 'name', 'owner', 'package']

# describe the return type of our user-defined function
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
resultType = StructType([StructField(f, StringType(), nullable=True) for f in interesting_fields])

# this is the body of our first user-defined function, to restrict us to a subset of fields
def trimFieldImpl(js):
    try:
        d = json.loads(js)
        return [d.get(f) for f in interesting_fields]
    except:
        # return an empty struct if we fail to parse this message
        return [None] * len(interesting_fields)
    
from pyspark.sql.functions import udf

# register trimFieldImpl as a user-defined function
trimFields = udf(trimFieldImpl, resultType)

trimmedDF = df.withColumn("msg", trimFields("msg"))

In [ ]:
trimmedDF.printSchema()

Data frames are a great way to explore structured data, but you can also train models against them (either by converting to RDDs and using [MLlib](https://spark.apache.org/docs/latest/mllib-guide.html) or by using [ML Pipelines](https://spark.apache.org/docs/latest/ml-guide.html) to define learning pipelines directly on data frames).

Let's extract bug and update comments from our `fedmsg` data and use those to train a word2vec model.

In [ ]:
def getComments(js):
    try:
        # construct a dict from the json string
        # we care about the following paths:  
        # * /comment/text 
        # * /notes/
        # * /update/comments//text
        d = json.loads(js)
        comment = [d.get('comment', {})]
        notes = d.get('notes', [])
        update_comments =  d.get('update', {}).get('comments', [])
        comment_texts = [c['text'] for c in comment + update_comments if 'text' in c]
        return comment_texts + notes
    except:
        return[]

commentsRDD = msgRDD.flatMap(lambda js: getComments(js))

# Turn comments into sequences of words.  Convert everything 
# to lowercase first to avoid spurious "synonyms" between differently-
# capitalized words (but try this also without `w.lower()` and see how 
# your results change!)
#
# We won't bother stripping punctuation or stemming but while we're in
# #YOLO territory for a tutorial and demo, you'd surely want to do 
# something more sensible in a real application.
wordSeqs = commentsRDD.map(lambda s: [w.lower() for w in s.split()])

In [ ]:
# actually train a model

from pyspark.mllib.feature import Word2Vec

w2v = Word2Vec()
model = w2v.fit(wordSeqs)

# find synonyms for a given word
synonyms = model.findSynonyms('works', 5)

for word, distance in synonyms:
    print("{}: {}".format(word, distance))

In [ ]:
# see some of the words in the model

list(model.getVectors().keys())[:20]

# Some exercises:

1.  Try running the word2vec pipeline above without converting words to lowercase (you can eliminate the list comprehension or change `w.lower()` to `w`).  How does this change your results?
2.  Try eliminating punctuation from words (use a regular expression).  You may also want to remove _stopwords_, or extremely common words (like articles, prepositions, conjunctions, etc.), and the `StopWordsRemover` class in Spark will remove stopwords from prose in several languages.  Check the documentation for details on how to use it (you'll want to use the MLlib version and not the `spark.ml` version since we're working with RDDs and not data frames).  Run your model and some queries after each step.  Do the additional data cleaning steps improve your results?  If not, why do you suppose the results change?
3.  You may also be interested in stemming your words (e.g. converting "works", "worked", "working", "worker" to "work").  While you can also use a regular expression for this, you may find it's more productive to use an external library.  Consider using the [NLTK](http://nltk.org) package, which provides a sensible text tokenizer and a stemmer, among other tools for natural language processing.
4.  The synonym query is interesting, but the word2vec model also supports finding analogies through linear transformations, for example, in a word2vec model trained on general English text, the vector for "king" plus the vector for "woman" minus the vector for "man" will be very similar to the vector for "queen".  While the training corpus we have with a subset of fedmsg messages is far, far, smaller than we'd want to use for a general model, you may want to play with this and see how it works.  You can use `getVectors()` to get the vectors for particular words, `numpy` for the vector transformations, and then you can pass a vector to the `findSynonyms()` method to see which words have vectors closest to a given vector.